<a href="https://colab.research.google.com/github/SamyaAmernis/DRiM-Challenge/blob/main/modele_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df_creation = pd.read_csv(filepath_or_buffer = './TABLE_CREATION_VF.csv')
df_defaut = pd.read_csv(filepath_or_buffer = './TABLE_DEFAUT_VF.csv')
df_pib = pd.read_csv(filepath_or_buffer = './TABLE_PIB_CLEAN.csv')
df_chomage = pd.read_csv(filepath_or_buffer = './TABLE_CHOMAGE_VF.csv')



FileNotFoundError: ignored

In [ ]:
import pandas as pd
from unidecode import unidecode

<h3>DISPLAY INITIAL DB

In [ ]:
df_chomage

In [ ]:
df_defaut

In [ ]:
df_pib

In [ ]:
df_chomage

 <h3> 1. Faire la fusion PIB CHOMAGE

In [ ]:
df_chomage = df_chomage.rename(columns={'Libellé': 'Région'})

In [ ]:
df_chomage

In [ ]:
df_pib['Région'] = df_pib['Région'].apply(lambda x: unidecode(x).upper())

In [ ]:
df_pib

<h4 style="color:red"> Attention la clé primare est région & année <h4>

In [ ]:
df_merged2 = pd.merge(df_pib, df_chomage, left_on='Région', right_on='Région')
df_merged2

Avec cette clé c'est uniquement le PIB de 2012 qui est appliqué à toutes les années...

On change le titre trimestre en Année dans la base chômage.

In [ ]:
df_chomage = df_chomage.rename(columns={'trimestre': 'Année'})

In [ ]:
df_chomage

On merge avec la clé primaire

In [ ]:
df_pib.info()

In [ ]:
df_chomage.info()

<p style="color:red"> Il faut mettre l'année en INT64 car les valeurs des PIB ont été importés comme des string au lieu de INT <p>

In [ ]:
df_pib['Année'] = df_pib['Année'].str.replace('\D', '', regex=True)
df_pib['Année'] = df_pib['Année'].astype('int64')

In [ ]:
df_pib.info()

In [ ]:
df_pib['PIB_VAL'] = df_pib['PIB_VAL'].str.replace('\D', '', regex=True)
df_pib['PIB_VAL'] = df_pib['PIB_VAL'].astype('int64')

In [ ]:
df_pib

In [ ]:
df_pib['PIB_VOL'] = df_pib['PIB_VOL'].str.replace('\D', '', regex=True)
df_pib['PIB_VOL'] = df_pib['PIB_VOL'].astype('int64')

In [ ]:
df_pib['PIB_HAB'] = df_pib['PIB_HAB'].str.replace('\D', '', regex=True)
df_pib['PIB_HAB'] = df_pib['PIB_HAB'].astype('int64')

In [ ]:
df_pib['PIB_EMP'] = df_pib['PIB_EMP'].str.replace('\D', '', regex=True)
df_pib['PIB_EMP'] = df_pib['PIB_EMP'].astype('int64')

In [ ]:
df_pib

In [ ]:
df_merged = pd.merge(df_pib, df_chomage, on=['Région', 'Année'])

In [ ]:
df_merged

In [ ]:
df_code_84 = df_merged.loc[df_merged['Code'] == 84]
df_code_84

 <h3> 2. Faire la fusion MERGE 1 wth CREATION & Defaut
   

In [ ]:
df_creation

<h3 style="color:blue" echelle departementale

In [ ]:
df_creation = df_creation.rename(columns={'trimestre': 'Année'})
df_creation = df_creation.rename(columns={'code_region': 'Code'})
df_creation

<h3 style="color:blue"> Hypothèse 1 Echelle departementale

In [ ]:
df_merged_h1 = pd.merge(df_creation, df_merged,  on=['Code', 'Année'] ,  how='left')
df_merged_h1

In [ ]:
df_code_84 = df_merged.loc[df_merged['Code'] == 4]
df_code_84

In [ ]:
df_merged_h1.info()

In [ ]:
df_merged_h1 = df_merged_h1.dropna(subset=['Région'])

In [ ]:
df_merged_h1

In [ ]:
df_code_84_h1 = df_merged_h1.loc[df_merged_h1['Code'] == 84]
df_code_84_h1

<h4>Ajout des defauts<h4>

In [ ]:
df_defaut

In [ ]:
df_defaut = df_defaut.rename(columns={'code_region': 'Code'})
df_defaut = df_defaut.rename(columns={'trimestre': 'Année'})
df_defaut

In [ ]:
df_merged_2_h1 = pd.merge(df_merged_h1, df_defaut,  on=['Secteur','Code', 'Année', 'code_dep'] ,  how='left')
df_merged_2_h1

In [ ]:
df_merged_2_h1.info()

In [ ]:
df_merged_2_h1['nombre_defaut'] = df_merged_2_h1['nombre_defaut'].fillna(0)

In [ ]:
df_merged_2_h1

In [ ]:
df_merged_3_h1 = df_merged_2_h1

In [ ]:
df_merged_3_h1['variation'] = df_merged_3_h1['nombre_creation'] - df_merged_3_h1['nombre_defaut']

In [ ]:
df_merged_3_h1

<p style="color:red"> On ne peut faire notre modèle linéaire que sur des variables numériques il faut donc segmenter nos regions <p>

In [ ]:
df_creation

In [ ]:
df_creation_w = df_creation[["Code","nombre_creation"]]

In [ ]:
df_creation_w

In [ ]:
df_grouped = df_creation_w.groupby(['Code'])['nombre_creation'].sum().reset_index()

In [ ]:
df_grouped

In [ ]:
df_grouped['Quartile_Region_MOY'] = pd.qcut(df_grouped['nombre_creation'], q=4, labels=[1, 2, 3, 4])
df_grouped

In [ ]:
df_creation2 = df_creation.merge(resultat[['Secteur', 'Quartile_MOY']], on='Secteur', how='left')

In [ ]:
df_merged_4_h1 = df_merged_3_h1.merge(df_grouped[['Code', 'Quartile_Region_MOY']], on='Code', how='left')
df_merged_4_h1

In [ ]:
df_merged_4_h1.info()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:

# Sélectionnez les colonnes pour X (caractéristiques) et y (variable cible)
X = df_merged_4_h1[[ 'Année', 'Quartile_MOY', 'Quartile_CV', 'PIB_VAL', 'PIB_VOL', 'chomage', 'Quartile_Region_MOY']]
y = df_merged_4_h1['variation']

# Convertissez les variables catégorielles en variables indicatrices (dummies)
#X = pd.get_dummies(X, drop_first=True)

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




In [ ]:
# Initialisez le modèle de régression linéaire
model = LinearRegression()

# Entraînez le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)


In [ ]:
# Faites des prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluez la performance du modèle
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Affichez les performances du modèle
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

In [ ]:
# Obtenez les coefficients du modèle
coefficients = model.coef_

# Obtenez l'intercept du modèle
intercept = model.intercept_

# Affichez les coefficients
print('Coefficients:', coefficients)
print('Intercept:', intercept)

In [ ]:
import statsmodels.api as sm

# Ajoutez une colonne constante pour représenter l'intercept dans le modèle
X_train = sm.add_constant(X_train)

# Initialisez le modèle de régression linéaire avec statsmodels
model_stats = sm.OLS(y_train, X_train)

# Ajustez le modèle
result = model_stats.fit()

# Affichez les statistiques détaillées
print(result.summary())

L'output fournit les résultats de la régression linéaire. Voici comment interpréter les principaux éléments de la sortie :

Coefficients (coef):

const (constante) : -2.89e+04
Année : 14.1580
Quartile_MOY : 85.1934
Quartile_CV : 10.6791
PIB_VAL : 0.0041
PIB_VOL : -0.0037
chomage : 8.8937
Quartile_Region_MOY_2 : 38.3696
Quartile_Region_MOY_3 : 35.7284
Quartile_Region_MOY_4 : 1.9070
Chaque coefficient est associé à une variable explicative particulière. Par exemple, pour la variable 'Année', une augmentation d'une unité de l'année est associée à une augmentation d'environ 14.16 unités dans la variable dépendante 'variation'. Les valeurs 'const' représentent l'intercept du modèle.

P>|t| (p-value):

La colonne P>|t| donne la valeur p associée à chaque coefficient. Si la valeur p est inférieure à un niveau de signification choisi (par exemple, 0.05), le coefficient est généralement considéré comme significatif. Dans votre sortie, tous les coefficients, à l'exception de 'Quartile_Region_MOY_4', sont significatifs.
R-squared (R²) :

R-squared mesure la proportion de la variance dans la variable dépendante qui est expliquée par le modèle. Dans ce cas, le R-squared est de 0.274, ce qui signifie que le modèle explique environ 27.4% de la variance dans 'variation'.
Adj. R-squared (R² ajusté) :

C'est une version ajustée du R-squared qui prend en compte le nombre de variables explicatives dans le modèle. Un R-squared ajusté plus proche de 1 est généralement préférable.
F-statistic :

L'analyse de la variance (ANOVA) teste l'hypothèse nulle que tous les coefficients du modèle sont égaux à zéro. Le F-statistic est utilisé pour tester cette hypothèse. Une valeur faible de Prob (F-statistic) indique que le modèle dans son ensemble est significatif. Ici, le Prob (F-statistic) est proche de zéro, ce qui indique que le modèle est globalement significatif.
Prob (Omnibus) :

Test d'omnibus : teste l'hypothèse nulle que les résidus sont normalement distribués. Une faible valeur p suggère que la distribution des résidus n'est pas normale.
Durbin-Watson :

Mesure de l'autocorrélation des résidus. La valeur idéale est proche de 2. Une valeur entre 1.5 et 2.5 est généralement acceptable.
Jarque-Bera (JB) :

Un autre test de normalité des résidus. Si la valeur p est faible, cela suggère que les résidus ne sont pas normalement distribués.
Skew et Kurtosis :

Mesures de la symétrie et de l'aplatissement de la distribution des résidus.
Condition Number :

Mesure la sensibilité du modèle aux petites modifications des données. Une valeur élevée suggère une forte multicollinéarité.
Il est important de considérer ces éléments dans le contexte de votre problème et de comprendre les implications pratiques des résultats

In [ ]:
df_merged_4_h1

<p style="color:red"> On n'a pas un modèle satisfaisant car on a une segmentation spatiale sur les régions et des observations sur les départements, or on peut avoir une grande variabilité pour les département au sein d'une région  

In [ ]:
df_merged_4_h1.info()

In [ ]:
df_merged_1_h2 = df_merged_4_h1

In [ ]:
df_merged_1_h2['Quartile_Region_MOY'] = df_merged_1_h2['Quartile_Region_MOY'].astype('int64')
df_merged_1_h2

In [ ]:
df_merged_1_h2.info()

In [ ]:
df_merged_2_h2= df_merged_1_h2.drop(['code_dep', 'nombre_creation', 'nombre_defaut'], axis=1)
df_merged_2_h2

In [ ]:
df_merged_3_h2 = df_merged_2_h2.groupby(['Secteur', 'Année', 'Code', 'Région'])
df_merged_3_h2

In [ ]:
df_means = df_merged_3_h2[['Quartile_MOY', 'Quartile_CV','Quartile_Region_MOY', 'chomage', 'PIB_HAB', 'PIB_EMP']].mean()

In [ ]:
df_sums = df_merged_3_h2[['PIB_VAL', 'PIB_VOL', 'variation']].sum()

In [ ]:
df_final = pd.merge(df_means, df_sums, left_index=True, right_index=True)

In [ ]:
df_final

In [ ]:
df_final.reset_index(inplace=True)

In [ ]:
df_final

In [ ]:

# Sélectionnez les colonnes pour X (caractéristiques) et y (variable cible)
X = df_final[[ 'Année', 'Quartile_MOY', 'Quartile_CV', 'PIB_VAL', 'PIB_VOL', 'chomage', 'Quartile_Region_MOY']]
y = df_final['variation']

# Convertissez les variables catégorielles en variables indicatrices (dummies)
#X = pd.get_dummies(X, drop_first=True)

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




In [ ]:
# Initialisez le modèle de régression linéaire
model = LinearRegression()

# Entraînez le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)


In [ ]:
# Faites des prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluez la performance du modèle
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Affichez les performances du modèle
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

In [ ]:
# Obtenez les coefficients du modèle
coefficients = model.coef_

# Obtenez l'intercept du modèle
intercept = model.intercept_

# Affichez les coefficients
print('Coefficients:', coefficients)
print('Intercept:', intercept)

In [ ]:
import statsmodels.api as sm

# Ajoutez une colonne constante pour représenter l'intercept dans le modèle
X_train = sm.add_constant(X_train)

# Initialisez le modèle de régression linéaire avec statsmodels
model_stats = sm.OLS(y_train, X_train)

# Ajustez le modèle
result = model_stats.fit()

# Affichez les statistiques détaillées
print(result.summary())

In [ ]:

# Sélectionnez les colonnes pour X (caractéristiques) et y (variable cible)
X = df_final[[ 'Année', 'Quartile_MOY', 'Quartile_CV', 'chomage', 'Quartile_Region_MOY']]
y = df_final['variation']

# Convertissez les variables catégorielles en variables indicatrices (dummies)
#X = pd.get_dummies(X, drop_first=True)

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialisez le modèle de régression linéaire
model = LinearRegression()

# Entraînez le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Faites des prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluez la performance du modèle
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Affichez les performances du modèle
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

In [ ]:
# Obtenez les coefficients du modèle
coefficients = model.coef_

# Obtenez l'intercept du modèle
intercept = model.intercept_

# Affichez les coefficients
print('Coefficients:', coefficients)
print('Intercept:', intercept)

In [ ]:
import statsmodels.api as sm

# Ajoutez une colonne constante pour représenter l'intercept dans le modèle
X_train = sm.add_constant(X_train)

# Initialisez le modèle de régression linéaire avec statsmodels
model_stats = sm.OLS(y_train, X_train)

# Ajustez le modèle
result = model_stats.fit()

# Affichez les statistiques détaillées
print(result.summary())